In [1]:
import numpy as np

def read_input(file_path):
    """Reads input data from the specified file and returns parsed values."""
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
    except FileNotFoundError:
        raise FileNotFoundError(f"The file {file_path} does not exist.")

    try:
        # Extract values from the second line, splitting by whitespace and converting to appropriate types
        values = lines[1].strip().split()
        DNCIL = int(float(values[0]))
        DNP = int(float(values[1]))
        DNIN = int(float(values[2]))

        RC = []
        MU = []
        for line in lines[3:3 + DNCIL]:
            r, m = map(float, line.split())
            RC.append(r)
            MU.append(m)

    except (IndexError, ValueError) as e:
        raise ValueError(f"Error parsing the input file: {e}")

    return DNCIL, DNP, DNIN, np.array(RC), np.array(MU)

def calculate_trasm(DNCIL, DNP, DNIN, RC, MU):
    """Calculates the TRASM value based on the given parameters."""
    if DNP > 199999:
        raise ValueError("DNP cannot be greater than 199999.")

    # Ensure NP is odd
    NP = DNP if DNP % 2 != 0 else DNP + 1
    NPM = NP - 1
    DX = RC[DNIN - 1] / NPM

    # Calculate RC squared values
    RC2 = RC ** 2

    F = np.zeros(NP)
    for IX in range(NP):
        X = DX * (IX - 1) - 1.e-06
        X2 = X * X
        Y = np.where(X < RC, np.sqrt(RC2 - X2), 0.0)
        
        XE = np.dot(np.diff(Y, prepend=0), MU)
        XE *= 2.0

        F[IX] = np.exp(-XE)

    TRASM = F[0] + F[NPM] + 4.0 * F[NPM]

    # Correct the summation ranges to ensure proper broadcasting
    TRASM += np.sum(4.0 * F[1:NPM:2]) + np.sum(2.0 * F[2:NPM-1:2])
    TRASM = TRASM * DX / (3.0 * RC[DNIN - 1])

    return TRASM

def main():
    """Main function to read input data and calculate TRASM."""
    file_path = './TrasbIN.txt'
    DNCIL, DNP, DNIN, RC, MU = read_input(file_path)
    TRASM = calculate_trasm(DNCIL, DNP, DNIN, RC, MU)
    print(f"TRASM = {TRASM:.4f}")

if __name__ == "__main__":
    main()


/tmp/ipykernel_15852/2680518683.py:47: RuntimeWarning: invalid value encountered in sqrt
  Y = np.where(X < RC, np.sqrt(RC2 - X2), 0.0)


TRASM = 0.8009
